In [1]:
%load_ext autoreload
%autoreload 2

import sys
import time
import dask.dataframe as dd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

sys.path.append('/root/code/elo/Tools/')
from tools import *

In [2]:
new = pd.read_csv('/root/data/new_merchant_transactions.csv')
his = pd.read_csv('/root/data/historical_transactions.csv')

In [3]:
new.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [4]:
his.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [5]:
new.shape

(1963031, 14)

# NaN data

In [6]:
for i in new.columns:
    print(f'{i} : {sum(new[i].isnull())}')

authorized_flag : 0
card_id : 0
city_id : 0
category_1 : 0
installments : 0
category_3 : 55922
merchant_category_id : 0
merchant_id : 26216
month_lag : 0
purchase_amount : 0
purchase_date : 0
category_2 : 111745
state_id : 0
subsector_id : 0


In [7]:
for i in his.columns:
    print(f'{i} : {sum(his[i].isnull())}')

authorized_flag : 0
card_id : 0
city_id : 0
category_1 : 0
installments : 0
category_3 : 178159
merchant_category_id : 0
merchant_id : 138481
month_lag : 0
purchase_amount : 0
purchase_date : 0
category_2 : 2652864
state_id : 0
subsector_id : 0


# authorized_flag

In [ ]:
new.authorized_flag.value_counts()

In [ ]:
his.authorized_flag.value_counts().compute()

In [ ]:
his[his.authorized_flag == 'N'].head()

# card_id

In [ ]:
new.card_id.nunique()

In [ ]:
his.card_id.nunique().compute()

# city_id

In [ ]:
new.city_id.hist(bins=40, color='lightgray', figsize=[10,5])
plt.title('CityId hist',size=16)
plt.show()

In [ ]:
new.city_id.dtype

**city_id:** 可以看出这是一个离散的，类别化的特征，如果我们不使用基于距离的算法，那么便不需要将其onehot编码,保留即可。

# category_1

In [ ]:
new.category_1.value_counts()

In [ ]:
his.category_1.value_counts().compute()

**category_1：**这是一个bool类型特征，just 0,1 化处理

# installments

In [ ]:
new.installments.value_counts()

In [ ]:
his.installments.value_counts().compute()

**installments:** ，这是付款分了多少期的 feature ，建议保留

# category_3

In [ ]:
new.category_3.value_counts()

In [ ]:
his.category_3.value_counts().compute()

In [ ]:
print(f'category_3 NaN data sum is: {sum(new.category_3.isnull())}')
new.category_3.fillna('nan', inplace=True)

category_3:建议将nan作为一个类别,然后将其数值化。

# merchant_category_id

In [ ]:
new.merchant_category_id.hist(bins=30, figsize=[10,5], color='lightgray')
plt.title('merchant_category_id', size=16)
plt.show()

**merchant_category_id:** 保留

# merchant_id

In [ ]:
print(f'merchant_id number is:{new.merchant_id.nunique()}')

**merchant_id:** 是与该账号进行交易的商家的id

# month_lag

In [ ]:
new.month_lag.value_counts()

In [ ]:
his.month_lag.value_counts().compute()

**new.month_lag:** 经过验证，该特征指的是从该购买日到2018年2月之间的月份差。在该日期以后的交易被归到new 在之前的交易被归到his。

# purchase_amount

In [ ]:
idx = (new.purchase_amount>-1) & (new.purchase_amount<1)
new.purchase_amount[idx].hist(bins=50, color='lightgray', figsize=[10,5])
plt.title('purchase_amount')
plt.show()

In [ ]:
idx = (new.purchase_amount>-1) & (new.purchase_amount<1)
new.purchase_amount[idx].apply(lambda x: np.log(x+0.75)).hist(bins=50, color='lightgray', figsize=[10,5])
plt.title('purchase_amount')
plt.show()

这个应该指的是交易的金额，但是这个交易金额经过了某种处理，我们试试能不能将其恢复一下

In [ ]:
idx = (his.purchase_amount>-1) & (his.purchase_amount<1)
his.purchase_amount[idx].compute().hist(bins=50, color='lightgray', figsize=[10,5])
plt.title('purchase_amount')
plt.show()

# purchase_date

In [ ]:
new['purchase_date'] = pd.to_datetime(new.purchase_date)

In [ ]:
print(new.purchase_date.min(),' to ',new.purchase_date.max())

In [ ]:
his['purchase_date'] = dd.to_datetime(his.purchase_date)

In [ ]:
print(his.purchase_date.min().compute(),' to ', his.purchase_date.max().compute())

#### q:new 与 his 在时间上的联系

In [ ]:
new[new.card_id == 'C_ID_4e6213e9bc']

In [ ]:
his[his.card_id == 'C_ID_4e6213e9bc'].compute()

In [ ]:
his[(his.authorized_flag == 'N') & (his.card_id == 'C_ID_4e6213e9bc')].compute()

In [ ]:
train = pd.read_csv('/home/leechh/data/elo/train.csv')
test = pd.read_csv('/home/leechh/data/elo/test.csv')

In [ ]:
test[test.card_id == 'C_ID_4e6213e9bc']

In [ ]:
train[train.card_id == 'C_ID_4e6213e9bc']

In [ ]:
我们可以看到，new

# category_2

In [ ]:
new.category_2.value_counts()

In [ ]:
his.category_2.value_counts().compute()

In [ ]:
sum(new.category_2.isnull())

In [ ]:
sum(his.category_2.isnull().compute())

onehot

# state_id

In [ ]:
new.state_id.nunique()

In [ ]:
his.state_id.nunique().compute()

In [ ]:
val = new.state_id.value_counts()
plt.figure(figsize=[10,5])
plt.title('state_id')
plt.bar(val.index, val, color='lightgray')
plt.grid()
plt.show()

**state_id:** 该特征为州ID，可能的话，onehot

# subsector_id

In [ ]:
new.subsector_id.nunique()

In [ ]:
his.subsector_id.nunique().compute()

In [ ]:
val = new.subsector_id.value_counts()
plt.figure(figsize=[10,5])
plt.title('state_id')
plt.bar(val.index, val, color='lightgray')
plt.grid()
plt.show()

**subsector_id:** 这个应该是支付的类别。onehot